# Conference Travel Planning
## Decision Framing Fundamentals

**Student Name:** [Your Name]  
**Date:** [Date]

## Assignment Overview

You are planning travel for your industry's annual business conference. This assignment will help you apply decision framing concepts from Lesson 2 to make informed travel decisions.

**Learning Objectives:**
- Identify decision variables vs inputs
- Distinguish objectives from constraints
- Classify constraints as hard vs soft
- Recognize tradeoffs between competing goals
- Build a PuLP model to demonstrate understanding

In [21]:
# Install required packages (if needed in Colab)
# Skip this cell if running locally and packages are already installed
%pip install pulp pandas matplotlib -q


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
# Download data files from GitHub repository
# This cell downloads the CSV files needed for the assignment

import urllib.request

# Repository base URL
repo_base_url = "https://raw.githubusercontent.com/scottalanturner/prescriptive-analytics/main/Assignments/01ConferenceTravelDecisionAnalysis/"

# Download the data files
lodging_url = repo_base_url + "lodging_options.csv"
flights_url = repo_base_url + "flight_options.csv"

try:
    urllib.request.urlretrieve(lodging_url, "lodging_options.csv")
    urllib.request.urlretrieve(flights_url, "flight_options.csv")
    print("Data files downloaded successfully!")
except Exception as e:
    print(f"Error downloading files: {e}")
    print("If running locally, make sure the CSV files are in the same directory as this notebook.")

Data files downloaded successfully!


In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pulp import LpMinimize, LpMaximize, LpProblem, LpVariable, lpSum, value, LpStatus

print("Libraries imported successfully!")

Libraries imported successfully!


In [24]:
# Load the provided datasets
# Files should be downloaded from GitHub in the previous cell (for Colab)
# If running locally, ensure CSV files are in the same directory as this notebook
lodging_df = pd.read_csv('lodging_options.csv')
flights_df = pd.read_csv('flight_options.csv')

# Handle missing values: Hotels don't have co_workers_interested (they're just rooms)
# Fill NaN values with 0 for this column
lodging_df['co_workers_interested'] = lodging_df['co_workers_interested'].fillna(0).astype(int)

# Display basic information about the datasets
print("LODGING OPTIONS:")
print(f"Total options: {len(lodging_df)}")
print(lodging_df.head())

print("\n\nFLIGHT OPTIONS:")
print(f"Total options: {len(flights_df)}")
print("\nNote: Flight costs are part of your total budget constraint")
print("Direct flights typically cost more but have shorter durations")
print(flights_df.head())

# Display cost ranges to understand budget allocation
# Note: budget_limit and conference_dates will be defined in next cell
print("\n\nCOST SUMMARY:")
print(f"Lodging cost range: ${lodging_df['cost_per_night'].min():.2f} - ${lodging_df['cost_per_night'].max():.2f} per night")
print(f"Flight cost range: ${flights_df['cost'].min():.2f} - ${flights_df['cost'].max():.2f}")

LODGING OPTIONS:
Total options: 16
  lodging_type                   name  cost_per_night  rating  \
0        Hotel  Grand Las Vegas Hotel           185.0     4.7   
1        Hotel     Stratosphere Tower            95.0     3.6   
2        Hotel         Caesars Palace           320.0     4.8   
3        Hotel              MGM Grand           210.0     4.5   
4        Hotel        Excalibur Hotel            75.0     3.4   

   distance_to_venue_miles  amenities_score  max_occupancy  \
0                      0.2              8.5              2   
1                      1.8              6.2              2   
2                      0.5              9.5              2   
3                      0.3              8.8              2   
4                      0.4              5.8              2   

   co_workers_interested  
0                      0  
1                      0  
2                      0  
3                      0  
4                      0  


FLIGHT OPTIONS:
Total options: 20

No

### Stakeholder Scenario

**Read the stakeholder scenario below and interpret what they need:**

"Management wants good networking coverage and is cost-conscious. They value quality but understand budget constraints."

**Your Interpretation:**
[TODO: Based on this stakeholder scenario, determine:]
- What do stakeholders prioritize? (objectives)
- What are their requirements? (constraints)
- How many people should we send? (decision variable: choose 2, 3, or 4 based on stakeholder priorities)

In [25]:
# Fixed information provided
# NOTE: These values are fixed for all students to ensure fairness in grading
conference_dates = 3  # Number of nights
budget_limit = 1500  # Total budget in dollars (includes lodging + flights for ALL people)
# This budget limit is fixed - students interpret stakeholder needs to determine how to allocate it

# Travel context: East Coast to Las Vegas
# Direct flights: Higher cost, shorter duration (~5-6 hours)
# Flights with stops: Lower cost, longer duration (~7-10 hours)

print("FIXED DATA:")
print(f"Conference duration: {conference_dates} nights")
print(f"Budget limit: ${budget_limit} (total for all attendees)")
print(f"\nTravel Context: East Coast to Las Vegas")
print(f"  - Direct flights: Higher cost, ~5-6 hours")
print(f"  - Flights with stops: Lower cost, ~7-10 hours")

# Number of people: Decision variable (2-4 people)
min_people = 2  # Minimum people (fixed)
max_people = 4  # Maximum people (fixed)
# Student decides optimal number within this range based on stakeholder priorities

# TODO: Based on stakeholder scenario, define these:
# min_rating_threshold = ?  # Minimum acceptable rating (based on stakeholder quality needs - you interpret)
# max_distance = ?  # Maximum distance from venue (based on stakeholder convenience needs - you interpret)

print("\nSTAKEHOLDER-DEFINED CONSTRAINTS:")
print(f"Number of people: {min_people}-{max_people} (you decide optimal number)")
print(f"[TODO: Set min_rating_threshold and max_distance based on stakeholder scenario interpretation]")

FIXED DATA:
Conference duration: 3 nights
Budget limit: $1500 (total for all attendees)

Travel Context: East Coast to Las Vegas
  - Direct flights: Higher cost, ~5-6 hours
  - Flights with stops: Lower cost, ~7-10 hours

STAKEHOLDER-DEFINED CONSTRAINTS:
Number of people: 2-4 (you decide optimal number)
[TODO: Set min_rating_threshold and max_distance based on stakeholder scenario interpretation]


## Part 1: Base Assignment

### 1. Decision Statement

**Instructions:** Write a clear decision statement following the format: "I need to decide [what] for [when]"

**Your Decision Statement:**
[TODO: Write your decision statement here]

**Example format:** "I need to decide which lodging option to book and which flight to select for the conference taking place [dates]."

### 2. Decision Variables vs Inputs

**Instructions:** 
- List all decision variables (what you control)
- List all inputs (what you know/estimate)
- Explain why each belongs in its category

**Decision Variables (What you control):**
[TODO: List your decision variables here]
- Which lodging option to select
- Which flight option to select
- How many people to send (2, 3, or 4 - you decide optimal number based on stakeholder priorities)

**Inputs (What you know/estimate):**
[TODO: List your inputs here]
- Costs, ratings, distances from dataset
- Budget limit, conference dates
- Stakeholder priorities (interpreted from scenario - these inform your objectives/constraints)

**Explanation:**
[TODO: Explain why each item belongs in its category]
[TODO: Explain how you interpreted stakeholder needs to determine the number of people decision]

In [26]:
# Helper: Display data to help identify inputs
print("INPUTS FROM DATASET:")
print("\nLodging inputs available:")
print(lodging_df.columns.tolist())

print("\nFlight inputs available:")
print(flights_df.columns.tolist())

INPUTS FROM DATASET:

Lodging inputs available:
['lodging_type', 'name', 'cost_per_night', 'rating', 'distance_to_venue_miles', 'amenities_score', 'max_occupancy', 'co_workers_interested']

Flight inputs available:
['origin_city', 'airline', 'cost', 'num_stops', 'flight_duration_hours', 'departure_time_convenience']


### 3. Objectives vs Constraints

**Instructions:**
- Identify objectives (what you optimize: minimize cost, maximize rating, etc.)
- Identify constraints (budget limits, minimum ratings, etc.)
- Explain the distinction

**Objectives (What stakeholders want to optimize - based on your interpretation):**
[TODO: List your objectives here]
- Examples: Minimize total cost, maximize team size, maximize average rating, minimize total travel time
- [TODO: Explain how stakeholder scenario informed your choice of objectives]

**Constraints (What stakeholders require - based on your interpretation):**
[TODO: List your constraints here]
- Budget limit (hard constraint)
- Number of people: 2-4 (fixed range, you decide optimal number)
- Minimum rating threshold (based on stakeholder quality requirements - you interpret)
- Maximum distance (based on stakeholder convenience needs - you interpret)

**Explanation:**
[TODO: Explain the distinction between objectives and constraints]
[TODO: Explain how stakeholder scenario informed your constraint definitions]

### 4. Hard vs Soft Constraints

**Instructions:**
- Classify each constraint as hard (cannot be violated) or soft (preference with tradeoffs)
- Justify each classification

**Hard Constraints:**
[TODO: List hard constraints with justification]

**Soft Constraints:**
[TODO: List soft constraints with justification]

**Explanation:**
[TODO: Explain how soft constraints could be handled differently (e.g., with penalties)]

### 5. Tradeoff Analysis

**Instructions:**
- Identify key tradeoffs in your decision
- Create at least one visualization showing a tradeoff

**Key Tradeoffs Identified (Based on Stakeholder Priorities):**
[TODO: List tradeoffs, e.g., 
- Team size vs Total cost (2 vs 3 vs 4 people - more people = higher cost but better coverage/networking)
- Lodging: Cost vs Quality (ratings) - Airbnb often cheaper per person with more people
- Lodging: Cost vs Convenience (distance to venue)
- Flights: Cost vs Time (direct flights cost more but save ~2-4 hours per person)
- Overall: Total cost vs Total travel time vs Team size (choosing 2, 3, or 4 people)
- Individual rooms vs Shared accommodation (Airbnb becomes more cost-effective with more people)]

**Tradeoff 1: [Name]**
[TODO: Describe this tradeoff]

In [27]:
# TODO: Create a visualization showing a tradeoff
# Example: Cost vs Rating scatter plot

# Your code here
# Hint: Use matplotlib to create scatter plots, line plots, etc.
# Example structure:
# plt.scatter(x_data, y_data)
# plt.xlabel('...')
# plt.ylabel('...')
# plt.title('Tradeoff: ...')
# plt.show()

### 6. PuLP Model Implementation

**Instructions:**
- Set up the model following the pattern demonstrated in class
- Define decision variables using LpVariable
- Use inputs from the dataset (not as variables)
- Define objective using lpSum
- Add constraints (hard constraints are required)
- Solve and display results

In [28]:
# Create the optimization model
# TODO: Choose LpMinimize or LpMaximize based on your objective
model = LpProblem("Conference_Travel_Planning", LpMinimize)

# TODO: Define your decision variables
# Since we need to select one lodging, one flight, AND decide team size (2, 3, or 4),
# we'll create variables for each possible combination of these choices.
# This makes it easier to calculate costs correctly.

# Create variables for each combination: (lodging option, flight option, team size)
# Example: solution_vars[(0, 2, 3)] means "select lodging 0 AND flight 2 AND send 3 people"
solution_vars = {}
for i in range(len(lodging_df)):
    for j in range(len(flights_df)):
        for k in [2, 3, 4]:  # Team sizes: 2, 3, or 4 people
            solution_vars[(i, j, k)] = LpVariable(f"lodging_{i}_flight_{j}_people_{k}", cat='Binary')

# Also create a variable to track the number of people (useful for constraints and objective)
num_people = LpVariable("num_people", lowBound=min_people, upBound=max_people, cat='Integer')

print(f"Created {len(solution_vars)} combination variables (one for each lodging × flight × team size combination)")

Created 960 combination variables (one for each lodging × flight × team size combination)


In [29]:
# TODO: Define your objective function
# Use lpSum to combine variables with their coefficients
# Examples:
#   To maximize team size: model += -num_people, "Maximize_Team_Size"  (minimize negative = maximize)
#   To minimize cost: model += total_cost, "Minimize_Cost"
#   To maximize rating: model += -selected_rating, "Maximize_Rating"

# Your objective here:
# Based on your stakeholder interpretation, choose what to optimize
# model += [your objective expression], "Objective_Name"

In [30]:
# TODO: Add hard constraints

# Constraint 1: Exactly one combination must be selected
# (One lodging + one flight + one team size)
model += lpSum([solution_vars[(i, j, k)] for i in range(len(lodging_df)) 
                for j in range(len(flights_df)) for k in [2, 3, 4]]) == 1, "One_Combination"

# Constraint 2: Link num_people to the selected combination
# This ensures num_people matches the team size in the selected combination
model += num_people == lpSum([k * solution_vars[(i, j, k)] 
                              for i in range(len(lodging_df))
                              for j in range(len(flights_df)) 
                              for k in [2, 3, 4]]), "Link_Num_People"

# Constraint 3: Budget constraint
# Calculate costs for each combination and ensure total <= budget_limit
# Note: Hotels charge per room (2 people per room), Airbnbs charge per person
# For hotels, we'll approximate: (cost_per_night / 2) * nights * team_size
# For Airbnbs: cost_per_night * nights * team_size
lodging_costs = []
flight_costs = []
for i in range(len(lodging_df)):
    for j in range(len(flights_df)):
        for k in [2, 3, 4]:
            # Calculate lodging cost for this combination (k is the team size)
            if lodging_df.iloc[i]['lodging_type'] == 'Airbnb':
                # Airbnb: per person
                lodging_cost = lodging_df.iloc[i]['cost_per_night'] * conference_dates * k
            else:
                # Hotel: per room (2 people), approximate as per-person
                lodging_cost = (lodging_df.iloc[i]['cost_per_night'] / 2) * conference_dates * k
            lodging_costs.append(lodging_cost * solution_vars[(i, j, k)])
            
            # Calculate flight cost for this combination
            flight_cost = flights_df.iloc[j]['cost'] * k
            flight_costs.append(flight_cost * solution_vars[(i, j, k)])

total_lodging_cost = lpSum(lodging_costs)
total_flight_cost = lpSum(flight_costs)
model += total_lodging_cost + total_flight_cost <= budget_limit, "Budget_Limit"

# Constraint 4: Minimum rating constraint for selected lodging
# [TODO: Replace min_rating_threshold with your interpreted value from stakeholder scenario]
model += lpSum([lodging_df.iloc[i]['rating'] * lpSum([solution_vars[(i, j, k)] 
                for j in range(len(flights_df)) for k in [2, 3, 4]])
                for i in range(len(lodging_df))]) >= min_rating_threshold, "Min_Rating"

# Constraint 5: Maximum distance constraint for selected lodging
# [TODO: Replace max_distance with your interpreted value from stakeholder scenario]
model += lpSum([lodging_df.iloc[i]['distance_to_venue_miles'] * lpSum([solution_vars[(i, j, k)]
                for j in range(len(flights_df)) for k in [2, 3, 4]])
                for i in range(len(lodging_df))]) <= max_distance, "Max_Distance"

# Constraint 6: Airbnb occupancy limit
# If an Airbnb is selected, team size must not exceed its max_occupancy
for i in range(len(lodging_df)):
    if lodging_df.iloc[i]['lodging_type'] == 'Airbnb':
        max_occ = lodging_df.iloc[i]['max_occupancy']
        for j in range(len(flights_df)):
            for k in [2, 3, 4]:
                if k > max_occ:
                    # This combination is invalid (team size too large for this Airbnb)
                    model += solution_vars[(i, j, k)] == 0, f"Airbnb_Occupancy_{i}_{j}_{k}"

print("Hard constraints added successfully!")

NameError: name 'min_rating_threshold' is not defined

In [ ]:
# Solve the model
model.solve()

# Check solution status
status = LpStatus[model.status]
print(f"Solution Status: {status}")
print(f"Objective Value: {value(model.objective):.2f}")
print(f"Team Size: {int(value(num_people))} people")
print()

# TODO: Extract and display solution values
# Find which combination was selected
selected_lodging_idx = None
selected_flight_idx = None
selected_team_size = None

for (i, j, k), var in solution_vars.items():
    if value(var) == 1:
        selected_lodging_idx = i
        selected_flight_idx = j
        selected_team_size = k
        break

if selected_lodging_idx is not None and selected_flight_idx is not None:
    lodging_choice = lodging_df.iloc[selected_lodging_idx]
    flight_choice = flights_df.iloc[selected_flight_idx]
    team_size = selected_team_size
    
    # Calculate costs (for display - hotels vs Airbnbs calculated differently)
    if lodging_choice['lodging_type'] == 'Hotel':
        # Hotels: per room (2 people per room)
        num_rooms = (team_size + 1) // 2
        lodging_cost = lodging_choice['cost_per_night'] * conference_dates * num_rooms
    else:
        # Airbnbs: per person
        lodging_cost = lodging_choice['cost_per_night'] * conference_dates * team_size
    
    flight_cost = flight_choice['cost'] * team_size
    total_cost = lodging_cost + flight_cost
    cost_per_person = total_cost / team_size
    
    # TODO: Display your solution details
    print("=" * 70)
    print("SOLUTION SUMMARY")
    print("=" * 70)
    print(f"\nTeam Size: {team_size} people")
    print(f"\nLODGING SELECTED:")
    print(f"  Name: {lodging_choice['name']}")
    print(f"  Type: {lodging_choice['lodging_type']}")
    print(f"  Cost per night: ${lodging_choice['cost_per_night']:.2f}")
    print(f"  Rating: {lodging_choice['rating']:.1f}")
    print(f"  Distance to venue: {lodging_choice['distance_to_venue_miles']:.1f} miles")
    if lodging_choice['lodging_type'] == 'Airbnb':
        print(f"  Max occupancy: {lodging_choice['max_occupancy']} people")
    
    print(f"\nFLIGHT SELECTED:")
    print(f"  Origin: {flight_choice['origin_city']}")
    print(f"  Airline: {flight_choice['airline']}")
    print(f"  Cost per person: ${flight_choice['cost']:.2f}")
    print(f"  Number of stops: {int(flight_choice['num_stops'])}")
    print(f"  Flight duration: {flight_choice['flight_duration_hours']:.1f} hours")
    
    print(f"\nCOST BREAKDOWN:")
    print(f"  Lodging cost: ${lodging_cost:.2f}")
    print(f"  Flight cost: ${flight_cost:.2f}")
    print(f"  Total cost: ${total_cost:.2f}")
    print(f"  Budget limit: ${budget_limit:.2f}")
    print(f"  Budget remaining: ${budget_limit - total_cost:.2f}")
    print(f"  Cost per person: ${cost_per_person:.2f}")
    
    # TODO: Explain how this solution addresses stakeholder priorities
    print(f"\nHOW THIS SOLUTION ADDRESSES STAKEHOLDER PRIORITIES:")
    print(f"  [TODO: Add your explanation here]")
else:
    print("No solution found. Check constraints.")

## Part 2: Stakeholder Considerations

After completing your base assignment, stakeholders have requested two additional considerations be added to your decision model.

**Instructions:**
1. Select ONE consideration from the provided list
2. Create ONE industry-specific consideration
3. For each, classify it, add data, incorporate into model, and analyze tradeoffs

### Stakeholder Consideration 1: [Selected from List]

**Selected Option:** [Enter your selection from the list]

**Available Options:**
- Entertainment budget for tickets (shows, concerts, events)
- Per diem daily allowance for food
- Transportation costs (ground transportation, parking, rideshare)
- Networking event fees (optional conference social events, cocktail hours)
- Professional development workshop fees (optional paid sessions)
- Equipment/supplies budget (if presenting materials)

**Classification:**
- Variable/Input/Objective/Constraint: [TODO]
- Hard or Soft: [TODO]
- Justification: [TODO]

**Data Added:**
[TODO: Describe the data you're adding - costs, requirements, etc.]

**How it affects the decision:**
[TODO: Explain]

In [ ]:
# TODO: Update your PuLP model to incorporate Consideration 1
# NOTE: For demonstration purposes, you can use regular Python numbers to show examples
# The actual PuLP model update should be in a new model (or recreate the existing one)

# Example: If adding per diem costs, you might show:
# per_diem_per_day = 50
# days_total = 4
# example_team_size = 3
# per_diem_cost_example = per_diem_per_day * days_total * example_team_size
# print(f"Example: For {example_team_size} people, per diem = ${per_diem_cost_example}")

# TODO: Create updated model with Consideration 1
# You'll need to:
# 1. Create a new model (e.g., model_updated = LpProblem(...))
# 2. Use the same combination variable approach (solution_vars2)
# 3. Add the new costs to your budget constraint
# 4. Update other constraints as needed

# Your code here:

### Stakeholder Consideration 2: Industry-Specific

**Your Industry-Specific Consideration:** [TODO: Describe your consideration]

**Why it's relevant to your field:**
[TODO: Explain relevance]

**Classification:**
- Variable/Input/Objective/Constraint: [TODO]
- Hard or Soft: [TODO]
- Justification: [TODO]

**Data Added:**
[TODO: Describe the data you're adding]

**How it affects the decision:**
[TODO: Explain]

In [ ]:
# TODO: Update your PuLP model to incorporate Consideration 2
# NOTE: For demonstration purposes, you can use regular Python numbers to show examples
# The actual PuLP model update should incorporate this into your updated model

# Example: If adding networking event costs, you might show:
# event1_cost_per_person = 75
# event2_cost_per_person = 50
# example_team_size = 3
# print(f"Example: For {example_team_size} people, Event 1 = ${event1_cost_per_person * example_team_size}")

# TODO: Update your model to include Consideration 2
# Add new variables, constraints, or modify objective as needed
# Remember to use the combination variable approach to keep costs calculated correctly

# Your code here:

In [ ]:
# Solve the updated model
# TODO: Use your updated model variable name (e.g., model_updated.solve())
model.solve()

# Display updated solution
status_updated = LpStatus[model.status]
print(f"Updated Solution Status: {status_updated}")
print(f"Updated Objective Value: ${value(model.objective):.2f}")

# TODO: Extract and display updated solution
# Use the same approach as before: iterate through solution_vars to find selected combination
# Store values for comparison with original solution

# TODO: Compare with original solution
# Show how stakeholder considerations changed the decision
# Display side-by-side comparison of:
# - Team size (original vs updated)
# - Lodging choice (original vs updated)
# - Flight choice (original vs updated)
# - Total costs (original vs updated)

### Tradeoff Analysis: Stakeholder Considerations

**How did adding these considerations affect your decision?**
[TODO: Analyze the tradeoffs introduced by stakeholder considerations]

In [ ]:
# TODO: Create visualization comparing original vs updated solution
# Show tradeoffs introduced by stakeholder considerations

## Executive Summary

**Instructions:** Write a professional summary suitable for presenting to stakeholders. Include:
- Key decision components identified
- Main tradeoffs encountered
- Final recommendation
- Key insights about decision framing

---

### Key Decision Components

[TODO: Summarize the key variables, inputs, objectives, and constraints]

### Main Tradeoffs

[TODO: Summarize the main tradeoffs you identified and analyzed]

### Final Recommendation

[TODO: Present your final recommendation based on the model solution]

### Key Insights About Decision Framing

[TODO: Reflect on what you learned about decision framing through this assignment]

---
**End of Assignment**